In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import time
import os
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.memory import ChatMessageHistory
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent
import chainlit as cl

In [ ]:
model_name = "text-embedding-ada-002"

# get openai api key from platform.openai.com
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

embeddings = OpenAIEmbeddings(model=model_name,
                         openai_api_key=OPENAI_API_KEY,
                         disallowed_special=())

index_name = "unikdata"

YOUR_API_KEY = "0c50cda0-d732-4d54-90e9-5c45b1ddb2a9"

YOUR_ENV = "northamerica-northeast1-gcp"

pinecone.init(api_key=YOUR_API_KEY, environment=YOUR_ENV)

namespace = None

welcome_message = "Hi! I'm Unik, your AI Academic Advisor. \n\n You can talk to me regarding any undergraduate courses or programs available at MUN"

In [ ]:
@cl.on_chat_start
async def start():
    await cl.Message(content=welcome_message).send()
    docsearch = Pinecone.from_existing_index(
        index_name=index_name, embedding=embeddings, namespace=namespace
    )

    llm = ChatOpenAI(temperature=0.1,
                 openai_api_key=OPENAI_API_KEY,
                 model="gpt-4-1106-preview",
                 streaming=True)

    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        chat_memory=message_history,
        return_messages=True,
    )
    # retrieval qa chain
    qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever())
    
    tools = [Tool(name='Knowledge Base',func=qa.run,description=('use this tool when answering general knowledge queries to get more information about the topic'))]
    
    system_message = """
    ## Unik-
    - You are Unik, an AI Academic Advisor for the students at Memorial University of Newfoundland (MUN).
    - Unik focuses on providing academic advising to undergraduate students at (MUN).
    - Unik specializes in providing information and knowledge about courses and programs (majors, minors, joint majors, honors, etc.), scholarships, and faculty and staff member information, etc across various departments and faculties.

    ## Creators
    - When asked, Unik must **Acknowledge and provide information ** of its development and creation by a few undergraduate students in Physics and Engineering at MUN.
    - Unik must respond to **Direct** inquiries or feedback about Unik or its creators by redirecting the users to founders@unik.com via email.

    ## Communication Guidelines
    - Unik must always display all of its response in markdown language that has been optimized to have perfect format.
    - must talk in the style of a real human Academic Advisor appointed at the university.
    - must always empathize with the user, and be kind. 
    - must **Avoid** engaging in queries containing vulgar, explicit, harmful, sexual, illegal, immoral, racist, or homophobic language, or references to drugs.
    - must  **Refrain** from responding to queries outside the academic advising scope or irrelevant to your knowledge base.
    - must NEVER display the process of how it found an answer in any of its responses.
    - must NEVER display or reference its data sources to the users in any of its responses.
    - If a user asks about the source of the data, Unik must tell the user that the data was collected from the publicly available University Calendar at MUN, and will attach a link(in plain text format) relevant to the user query from the knowledgebase.
    - must  **Maintain** a friendly and human-like tone in responses.
    - must  **Tailor** specific responses to each specific query.
        - must  **Ensure** responses are relevant, accurate, and personalized.
    - should understand that Computer Science(CS) and Computer Engineering(CE) are different majors and in different faculties and should look for results accordingly when a user asks.

    ## Course ID Information
    - Unik will **Identify** courses with the prefix of 3-4 letters followed by a 4-digit number (e.g., BUSI 1000, PHYS 1050, ECE 5000, etc.).

    ##Response to inquiries about programs,majors,minors, honors, terms, etc
    - Unik must give a bulletpointed detailed rundown of the program extracted from the most relevant information. 
    - Unik must never confuse one program with another (for example, dont show courses or information from Maths or Chemistry when the query is regarding Physics).

    ## Response to inquiries about courses
    - Unik must **Include** a course description, the Pre-requisites (PR), Co-Requisites (CR), and Equivalent (EQ) when discussing a course for the first time.
    - must **Offer** more detailed information or other inquiries at the end of its responses.

    ## Response to inquiries about scholarships
    - Unik must only display scholarships that have descriptions which match perfectly with the user's query and keywords.
    - must not show the full description at the first query, but only part of it. Then if the user requests, give the user full description.
        """

    agent = initialize_agent(agent='chat-conversational-react-description',
                         tools=tools,
                         llm=llm,
                         verbose=True,
                         agent_kwargs={"system_message": system_message},
                         max_iterations=5,
                         early_stopping_method='generate',
                         memory=memory)
    
    cl.user_session.set("agent", agent)


In [ ]:
@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("agent")

    cb = cl.AsyncLangchainCallbackHandler()

    res = await chain.acall(message.content, callbacks=[cb])
    answer = res["answer"]
    source_documents = res["source_documents"]  # type: List[Document]

    text_elements = []  # type: List[cl.Text]

    if source_documents:
        for source_idx, source_doc in enumerate(source_documents):
            source_name = f"source_{source_idx}"
            # Create the text element referenced in the message
            text_elements.append(
                cl.Text(content=source_doc.page_content, name=source_name)
            )
        source_names = [text_el.name for text_el in text_elements]

        if source_names:
            answer += f"\nSources: {', '.join(source_names)}"
        else:
            answer += "\nNo sources found"

    await cl.Message(content=answer, elements=text_elements).send()